In [ ]:
import random

# Parâmetros do problema
b = 10  # capacidade da mochila
k = 3   # limite global de penalidades
alpha = 0.5  # parâmetro da RCL

In [ ]:
import random

# Parâmetros do problema
b = 10  # capacidade da mochila
k = 3   # limite global de penalidades
alpha = 0.5  # parâmetro da RCL

# Lista de itens: id, lucro, peso, conjuntos a que pertence
itens = [
    {"id": 1, "lucro": 10, "peso": 4, "conjuntos": [1]},
    {"id": 2, "lucro": 8,  "peso": 3, "conjuntos": [1]},
    {"id": 3, "lucro": 5,  "peso": 2, "conjuntos": [2]},
    {"id": 4, "lucro": 7,  "peso": 5, "conjuntos": [1, 2]},
    {"id": 5, "lucro": 6,  "peso": 3, "conjuntos": []}
]

# Conjuntos de penalidade (Ci): lista com info por conjunto
conjuntos = {
    1: {"itens": [1, 2, 4], "h": 1, "d": 3},
    2: {"itens": [3, 4], "h": 1, "d": 2}
}

# Estado atual da construção
solucao = []
peso_total = 0
violacoes = {i: 0 for i in conjuntos}
violacoes_totais = 0

# Lista de itens ainda não escolhidos
itens_restantes = itens.copy()

while itens_restantes:
    # Avalia os itens com base em lucro líquido / peso
    candidatos = []
    for item in itens_restantes:
        lucro = item["lucro"]
        penalidade_esperada = 0
        for ci in item["conjuntos"]:
            if sum(1 for j in solucao if j in conjuntos[ci]["itens"]) >= conjuntos[ci]["h"]:
                penalidade_esperada += conjuntos[ci]["d"]
        criterio = (lucro - penalidade_esperada) / item["peso"]
        candidatos.append((item, criterio))

    # Calcula f_max e f_min
    criterios = [c[1] for c in candidatos]
    f_max = max(criterios)
    f_min = min(criterios)
    limiar = f_min + alpha * (f_max - f_min)

    # Monta a RCL
    rcl = [c[0] for c in candidatos if c[1] >= limiar]
    if not rcl:
        break

    # Escolhe aleatoriamente um item da RCL
    escolhido = random.choice(rcl)

    # Verifica se o item cabe na mochila
    if peso_total + escolhido["peso"] > b:
        itens_restantes.remove(escolhido)
        continue

    # Simula violação nos conjuntos
    violacoes_sim = violacoes.copy()
    violacoes_total_sim = violacoes_totais
    for ci in escolhido["conjuntos"]:
        num_itens_Ci = sum(1 for j in solucao if j in conjuntos[ci]["itens"])
        if num_itens_Ci >= conjuntos[ci]["h"]:
            violacoes_sim[ci] += 1
            violacoes_total_sim += 1

    if violacoes_total_sim > k:
        itens_restantes.remove(escolhido)
        continue

    # Adiciona o item à solução
    solucao.append(escolhido["id"])
    peso_total += escolhido["peso"]
    violacoes = violacoes_sim
    violacoes_totais = violacoes_total_sim
    itens_restantes.remove(escolhido)

# Mostra a solução construída
print("Itens escolhidos:", solucao)
print("Peso total:", peso_total)
print("Violação total:", violacoes_totais)
print("Violação por conjunto:", violacoes)
